![image.png](attachment:image.png)


## Task:

* Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

* Read the file ( Present approach of reading the file )

* Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational     efficiency

* Perform basic validation on data columns : eg: remove special character , white spaces from the col name

* As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of   
  read and write file, column name in YAML

* Validate number of columns and column name of ingested file with YAML.

* Write the file in pipe separated text file (|) in gz format.

* Create a summary of the file:

    Total number of rows,

    total number of columns

    file size
    
# Data Ingestion sample code walkthrough

## 
  > Create a utility file
  
  > Config file creation
  
  > Data ingestion pipeline


In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


### Write YAML file

In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ";"
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Period
    - Data
    - Magnitude
    - Subject
    - Group
    - Series

Writing file.yaml


In [3]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [4]:
config_data['inbound_delimiter']

';'

In [5]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ';',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Period', 'Data', 'Magnitude', 'Subject', 'Group', 'Series']}

In [6]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("test_data.csv",delimiter=';')
df_sample.head()

,Period,Data,Magnitude,Subject,Group,Series
0,2001.03,2462.5,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
1,2002.03,17177.2,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
2,2003.03,22530.5,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
3,2004.03,28005.1,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
4,2005.03,30629.6,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual


In [7]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,Period,Data,Magnitude,Subject,Group,Series
0,2001.03,2462.5,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
1,2002.03,17177.2,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
2,2003.03,22530.5,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
3,2004.03,28005.1,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
4,2005.03,30629.6,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19235 entries, 0 to 19234
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Period     19235 non-null  float64
 1   Data       17470 non-null  float64
 2   Magnitude  19235 non-null  int64  
 3   Subject    19235 non-null  object 
 4   Group      19235 non-null  object 
 5   Series     19235 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 901.8+ KB


In [9]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [10]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['period', 'data', 'magnitude', 'subject', 'group', 'series'], dtype='object')
columns of YAML are: ['Period', 'Data', 'Magnitude', 'Subject', 'Group', 'Series']


In [11]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [12]:
df.columns

Index(['period', 'data', 'magnitude', 'subject', 'group', 'series'], dtype='object')

In [13]:
import dask.dataframe as dd
from pathlib import Path

In [14]:
%%time
data = dd.read_csv("test_data.csv")

CPU times: user 8.79 ms, sys: 3.89 ms, total: 12.7 ms
Wall time: 30.6 ms


In [15]:
df

,period,data,magnitude,subject,group,series
0,2001.03,2462.5,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
1,2002.03,17177.2,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
2,2003.03,22530.5,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
3,2004.03,28005.1,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
4,2005.03,30629.6,6,Electronic Card Transactions (ANZSIC06) - ECT,Total values - Electronic card transactions A/...,Actual
...,...,...,...,...,...,...
19230,2021.12,33.3,0,Electronic Card Transactions (ANZSIC06) - ECT,Electronic card transactions by mean and propo...,Actual
19231,2022.03,33.7,0,Electronic Card Transactions (ANZSIC06) - ECT,Electronic card transactions by mean and propo...,Actual
19232,2022.06,33.5,0,Electronic Card Transactions (ANZSIC06) - ECT,Electronic card transactions by mean and propo...,Actual
19233,2022.09,33.2,0,Electronic Card Transactions (ANZSIC06) - ECT,Electronic card transactions by mean and propo...,Actual


In [16]:
df.to_csv(r'test_data.csv', sep = '|', index=False) 

In [17]:
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,period|data|magnitude|subject|group|series
0,2001.03|2462.5|6|Electronic Card Transactions ...
1,2002.03|17177.2|6|Electronic Card Transactions...
2,2003.03|22530.5|6|Electronic Card Transactions...
3,2004.03|28005.1|6|Electronic Card Transactions...
4,2005.03|30629.6|6|Electronic Card Transactions...


In [18]:
df.shape

(19235, 1)

In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19235 entries, 0 to 19234
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   period|data|magnitude|subject|group|series  19235 non-null  object
dtypes: object(1)
memory usage: 150.4+ KB
